In [1]:
import pandas as pd
import numpy as np
import pickle
import sklearn
import tensorflow as tf
import torch
import torch.nn as nn
import torch.utils 
from torch.utils.data import Dataset,DataLoader
import torch.utils.data
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import regularizers
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import mnist


2024-07-15 19:19:02.847553: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-15 19:19:02.867492: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-15 19:19:02.872902: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-15 19:19:02.886851: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-15 19:19:03.653684: W tensorflow/compiler/tf2

In [2]:
# Definition of the Autoencoder model as a subclass of the TensorFlow Model class

class SimpleAutoencoder(Model):
	def __init__(self,latent_dimensions , data_shape):
		super(SimpleAutoencoder, self).__init__()
		self.latent_dimensions = latent_dimensions
		self.data_shape = data_shape

		# Encoder architecture using a Sequential model
		self.encoder = tf.keras.Sequential([
			layers.Flatten(),
			layers.Dense(latent_dimensions, activation='relu', dtype='float32'),
		])

		# Decoder architecture using another Sequential model
		self.decoder = tf.keras.Sequential([
			layers.Dense(tf.math.reduce_prod(data_shape), activation='sigmoid', dtype='float32'),
			layers.Reshape(data_shape)
		])

	# Forward pass method defining the encoding and decoding steps
	def call(self, input_data):
		encoded_data = self.encoder(input_data)
		decoded_data = self.decoder(encoded_data)
		return decoded_data


In [3]:
dataset_path = "dataset_amyloid.pickle"
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print(pd.__version__)
print(sklearn.__version__)

with open(dataset_path, 'rb') as f:
    X_train, y_train, X_test, y_test = pickle.load(f)
    
# Normalize the data to the range [0, 1]
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

# Reshape the data to include the channel dimension (necessary for some models)
X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


True
NVIDIA A100-SXM4-80GB
2.2.2
1.3.2
(155, 10193, 1)
(155,)
(35, 10193, 1)
(35,)


In [5]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Specifying the shape of the input data based on the testing dataset
input_data_shape = X_test.shape[1:]

# Specifying the dimensionality of the latent space
latent_dimensions = 64

# Creating an instance of the SimpleAutoencoder model
simple_autoencoder = SimpleAutoencoder(latent_dimensions, input_data_shape)
# simple_autoencoder = simple_autoencoder.to(device)

simple_autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

simple_autoencoder.fit(X_train, X_train,
                       epochs=5,
                       batch_size = 16,
                       shuffle=True,
                       validation_data=(X_test, X_test))


Epoch 1/5


ValueError: Exception encountered when calling SimpleAutoencoder.call().

[1mInvalid dtype: <property object at 0x7f3985124ef0>[0m

Arguments received by SimpleAutoencoder.call():
  • input_data=tf.Tensor(shape=(None, 10193, 1), dtype=float32)